<a href="https://colab.research.google.com/github/xinnazim/esaa/blob/main/%ED%95%B8%EC%A6%88%EC%98%A89%EC%9E%A5_%EB%B9%84%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH09. 비지도 학습

### 9.1 군집
* 비슷한 샘플을 구별해 하나의 클러스터로 할당하는 작업
* 레이블 없음

> ex. 고객분류(추천시스템), 데이터 분석, 차원축소, 이상치 탐치, 준지도 학습, 검색엔진, 이미지 분할

#### 9.1.1 K-평균

In [9]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [5]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters = k)
y_pred = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [6]:
y_pred

array([0, 4, 1, ..., 2, 1, 4], dtype=int32)

In [11]:
y_pred is kmeans.labels_

True

In [12]:
# 알고리즘이 찾은 센트로이드 5개 확인
kmeans.cluster_centers_

array([[-2.80037642,  1.30082566],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80389616,  1.80117999]])

In [13]:
# 새로운 샘플에 가까운 센트로이드 클러스터 할당
X_new = np.array([[0,2], [3,2], [-3,3], [-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

> * 하드 군집: 샘플을 하나의 클러스터에 할당
* 소프트 군집: 클러스터마다 샘플에 점수를 부여하는 것

In [15]:
# 샘플과 각 센트로이드 사이의 거리 반환
kmeans.transform(X_new)

array([[2.88633901, 0.32995317, 2.9042344 , 1.49439034, 2.81093633],
       [5.84236351, 2.80290755, 5.84739223, 4.4759332 , 5.80730058],
       [1.71086031, 3.29399768, 0.29040966, 1.69136631, 1.21475352],
       [1.21567622, 3.21806371, 0.36159148, 1.54808703, 0.72581411]])

K-Means 평균 알고리즘 작동 방식
*  랜덤으로 중심(센트로이드)을 할당
* 샘플에 레이블 할당
* 센트로이드 업데이트, 변화 없을 때까지 반복

In [23]:
# 센트로이드 초기화

good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

In [26]:
kmeans.fit(X)

KMeans(init=array([[-3,  3],
       [-3,  2],
       [-3,  1],
       [-1,  2],
       [ 0,  2]]),
       n_clusters=5, n_init=1)

In [27]:
# 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리: 이너셔
kmeans.inertia_

211.5985372581684

In [28]:
# 이너셔의 음숫값 반환, 큰 값이 좋음
kmeans.score(X)

-211.59853725816836

K-평균 속도 개선과 미니배치 K-평균
* 불필요한 계산 피함, 알고리즘 속도 높임

In [29]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters = 5)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5)

최적의 클러스터 개수 찾기
* 실루엣 점수= 실루엣 계수의 평균

In [30]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

### 9.1.2 k-평균의 한계
* 최적이 아닌 솔루션을 피하려면 알고리즘 여러 번 실행
* 클러스터 개수를 지정해야함
* 클러스터의 크기나 밀집도가 서로 다르거나 원형이 아닐 경우 잘 작동 X

### 9.1.3 군집을 사용한 이미지 분할
* 이미지 분할: 이미지를 세그먼트 여러개로 분할
* 시맨틱 분할: 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당

In [31]:
import os
import urllib.request

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# 무당벌레 이미지 다운로드

images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7f93e27396f0>)

In [32]:
# 색상분할
from matplotlib.image import imread
image = imread(os.path.join('images','unsupervised_learning','ladybug.png'))
image.shape

(533, 800, 3)

In [33]:
# 배열을 RGB 색상의 긴 리스트로 변환하고 k-means 활용해 클러스터로 색상 모음
X = image.reshape(-1,3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

### 9.1.4 군집 사용 전처리

In [34]:
# 데이터셋 불러오기
from sklearn.datasets import load_digits
X_digits,y_digits = load_digits(return_X_y=True)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [36]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [37]:
log_reg.score(X_test, y_test)

0.9666666666666667

In [38]:
#k-평균을 전처리 단계로 사용
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=50)),
    ('log_reg', LogisticRegression()),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [39]:
pipeline.score(X_train, y_train)

0.9844097995545658

In [40]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] END ...............................kmeans__n_clusters=2; total time=   0.8s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.6s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.6s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.6s
[CV] END ...............................kmeans__n_clusters=5; total time=   1.1s
[CV] END ...............................kmeans__n_clusters=5; total time=   1.4s
[CV] END ...............................kmeans_

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [41]:
grid_clf.best_params_
grid_clf.score(X_test, y_test)

0.96

### 9.1.5 군집을 사용한 준지도 학습
* 레이블 없는 데이터 많을 때 사용

In [42]:
# 레이블된 50개 샘플에 로지스틱 회귀 모델
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled],y_train[:n_labeled])

LogisticRegression()

In [43]:
log_reg.score(X_test, y_test)

0.8177777777777778

> 각 클러스터에서 센트로이드에 가장 가까운 이미지 찾음-> 대표 이미지

In [44]:
k=50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

In [45]:
y_train[representative_digit_idx]

array([6, 5, 7, 9, 0, 1, 4, 2, 8, 3, 4, 9, 9, 1, 6, 5, 5, 1, 1, 8, 1, 7,
       3, 0, 8, 3, 6, 9, 3, 8, 2, 4, 7, 7, 0, 4, 7, 6, 7, 2, 2, 5, 4, 3,
       0, 1, 8, 5, 2, 7])

In [49]:
y_representative_digits = np.array([6, 5, 7, 9, 0, 1, 4, 2, 8, 3, 4, 9, 9, 1, 6, 5, 5, 1, 1, 8, 1, 7,
       3, 0, 8, 3, 6, 9, 3, 8, 2, 4, 7, 7, 0, 4, 7, 6, 7, 2, 2, 5, 4, 3,
       0, 1, 8, 5, 2, 7])

In [50]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

0.8888888888888888

> 성능 높아짐

In [51]:
# 레이블 전파: 레이블을 동일한 클러스터에 있는 모든 샘플로 전파
y_train_propagated = np.empty(len(X_train),dtype=np.int32)
for i in range(k):
  y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [52]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test,y_test)

0.9133333333333333

센트로이드와 가까운 샘플의 20%에만 레이블 전파해보기

In [53]:
percentile_closest = 20
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
  in_cluster = (kmeans.labels_ == i)
  cluster_dist = X_cluster_dist[in_cluster]
  cutoff_distance = np.percentile(cluster_dist, percentile_closest)
  above_cutoff = (X_cluster_dist > cutoff_distance)
  X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [54]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test,y_test)

0.9

In [55]:
np.mean(y_train_partially_propagated==y_train[partially_propagated])

0.986013986013986

### 9.1.6 DBSCAN
* 밀집된 연속적 지역을 클러스터로 정의

In [56]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [57]:
dbscan.labels_

array([-1,  0,  5,  1,  2,  0,  3,  4,  3,  0,  0,  5,  4,  3,  5,  2,  6,
        3,  0,  6,  3,  5,  2,  5,  4,  1,  7,  4,  3,  8,  2,  8, -1,  8,
        2,  8,  4, -1,  8,  7,  3,  6,  2,  1,  2,  3,  2,  5,  0,  9,  4,
        5,  8, 10,  8,  0,  2, -1,  9,  3,  2, 10,  9, -1,  9,  3,  8, 10,
        3,  5,  0,  3,  8, -1,  4,  0,  7,  3,  5,  4,  9,  3,  6,  4,  3,
        0,  6,  6,  5,  3,  0,  3,  8,  2,  1,  3,  5,  0,  0,  3,  5,  3,
        2,  6,  4,  4, 10,  0,  2,  6,  2,  9,  2,  0,  2,  3,  3,  1,  0,
       11,  8,  5,  5,  0,  5,  3,  2,  0,  8,  9,  3,  3,  2,  2,  6,  0,
        4,  4,  2,  8,  3,  8,  3,  4, 10, 11,  5,  0, -1,  3,  5,  4,  7,
        5,  2, 10,  3, -1,  0,  8,  7, -1,  5,  2,  3, -1,  0,  6, -1,  0,
        4,  3,  1,  3, -1,  2,  3,  3,  2,  7,  5, -1,  2,  3, -1,  8,  0,
        2,  8,  8, -1,  7,  8,  9, -1,  3,  3,  6,  3,  1,  3,  4,  4,  3,
        9,  2, -1, -1, -1,  4,  1,  3,  2,  8,  3,  8,  2,  8,  6,  4,  4,
        2,  0,  5,  3,  8

In [58]:
# 핵심 샘플의 인덱스: core_sample_indices_
len(dbscan.core_sample_indices_)

809

In [59]:
dbscan.core_sample_indices_

array([  1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  33,  35,  36,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  54,  55,  56,  58,  59,  60,
        61,  62,  64,  65,  66,  67,  68,  69,  70,  71,  72,  74,  75,
        76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117,
       118, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 149, 150, 151, 153, 154, 155, 156, 158, 159, 160, 162,
       163, 166, 167, 169, 170, 171, 173, 175, 176, 177, 178, 179, 180,
       182, 183, 185, 187, 189, 191, 193, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 209, 210, 211, 212, 213, 214, 21

In [60]:
dbscan.components_

array([[ 1.32553619, -0.44236794],
       [-0.95168477,  0.24543154],
       [ 0.14034342,  0.08290975],
       ...,
       [ 1.61232161, -0.3313091 ],
       [-0.30299159,  0.88839814],
       [ 1.929103  ,  0.14551888]])

> 샘플에 대해 클러스터 예측 불가-> KNeighborsClassifier 훈련

In [61]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [62]:
X_new = np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
knn.predict(X_new)
knn.predict_proba(X_new)

array([[0.  , 0.5 , 0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.28, 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.5 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 ,
        0.  , 0.  ],
       [0.32, 0.  , 0.  , 0.54, 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.1 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.72, 0.28, 0.  , 0.  , 0.  ,
        0.  , 0.  ]])

In [63]:
# kneighbors(): 가장 가까운 이웃 k개의 거리와 인덱스 반환(열을 k개 가진 행렬 두개 반환)
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist > 0.2] = -1
y_pred.ravel()

array([-1, 10, 12, -1])

### 9.1.7 다른 군집 알고리즘
* 병합 군집
  - 반복마다 병합 군집은 인접한 클러스터 쌍을 연결
  - 쌍을 트리로 그리면 이진 트리 얻을 수 있음
  - 대규모 샘플과 클러스터에 잘 확장됨
* BIRCH
  - 대규모 데이터셋
  - 훈련 과정에서 새로운 샘플을 클러스터에 빠르게 할당할 수 있는 정보를 담은 트리 구조 생성
  - 트리에 모든 샘플 저장 X, 제한된 메모리 사용
* 평균-이동
  - 클러스터 내부 밀집도 불균형할 때 여러 개로 나눔
  - 대규모 데이터셋 부적합
* 유사도 전파
  - 투표 방식 사용
  - 대규모 부적합
* 스펙트럼 군집
  - 유사도 행렬을 받아 저차원 임베딩 생성
  - 저차원 공간에서 또 다른 군집 알고리즘 사용

## 9.2 가우시안 혼합
* 가우시안 혼합 모델 GMM: 샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델

In [64]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [65]:
gm.weights_

array([0.21411327, 0.1985313 , 0.58735544])

In [66]:
gm.means_

array([[ 1.71924659, -0.07014496],
       [-0.75769404,  0.54667323],
       [ 0.47728255,  0.2681875 ]])

In [67]:
gm.covariances_

array([[[ 0.0628522 ,  0.06842964],
        [ 0.06842964,  0.09014762]],

       [[ 0.04808982,  0.05832104],
        [ 0.05832104,  0.08749862]],

       [[ 0.1664156 , -0.09808607],
        [-0.09808607,  0.28564925]]])

> 기댓값 최대화 알고리즘 사용
* 기댓값 단계: 샘플을 클러스터에 할당
* 최대화 단계: 클러스터 업데이트

In [69]:
# 알고리즘 수렴 여부, 반복 횟수 확인
gm.converged_

True

In [70]:
gm.n_iter_

15

In [71]:
# predict(): 하드 군집, predict_proba(): 소프트 군집
gm.predict(X)

array([1, 0, 2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2,
       2, 2, 2, 1, 0, 2, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 1, 0, 2, 0, 2, 1,
       2, 2, 2, 2, 0, 0, 2, 1, 1, 2, 1, 0, 2, 1, 0, 2, 2, 2, 0, 2, 0, 2,
       1, 2, 2, 1, 0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0,
       2, 2, 0, 2, 1, 2, 1, 2, 2, 0, 0, 2, 1, 2, 2, 0, 2, 2, 2, 0, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 1, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 0, 1, 0, 2, 2, 2, 0,
       2, 1, 2, 2, 2, 2, 0, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2,
       2, 2, 0, 1, 2, 1, 2, 0, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 2, 2, 0, 2,
       2, 2, 2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2,
       2, 2, 0, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 2,

In [72]:
gm.predict_proba(X)

array([[1.69926900e-187, 9.99641634e-001, 3.58365893e-004],
       [8.50848352e-001, 7.82664210e-180, 1.49151648e-001],
       [1.41088199e-106, 1.19347585e-011, 1.00000000e+000],
       ...,
       [1.83366734e-152, 3.24141596e-001, 6.75858404e-001],
       [9.99655641e-001, 1.41554459e-206, 3.44358970e-004],
       [6.64770386e-137, 2.21252457e-003, 9.97787475e-001]])

In [73]:
X_new, y_new = gm.sample(6)
X_new

array([[ 1.82642548, -0.15275792],
       [ 1.22111326, -0.5108873 ],
       [-0.66015212,  0.6145284 ],
       [-0.69427407,  0.4080961 ],
       [ 0.64662615,  0.12765829],
       [ 0.67785556,  0.33451208]])

In [74]:
y_new

array([0, 0, 1, 1, 2, 2])

In [75]:
# pdf 로그 예측
gm.score_samples(X)

array([-1.19501896e+00, -1.09019899e+00, -1.49259489e+00, -4.14012602e-01,
       -1.39828656e+00, -2.09175889e+00, -1.73416646e+00, -1.78589807e+00,
       -1.63278597e+00, -5.00699827e-01, -6.86426973e-01, -1.72025227e+00,
       -1.74687128e+00, -1.74719980e+00, -1.54970578e+00, -1.49720908e+00,
       -9.30159623e-01, -1.29685575e+00, -6.52375386e-01, -7.56427951e-01,
       -1.54699018e+00, -1.70618804e+00, -1.48539461e+00, -1.69942598e+00,
       -1.67919663e+00, -9.90438135e-01, -9.95134401e-02, -1.72891170e+00,
       -1.65925440e+00, -7.18711071e-01, -1.69902483e+00, -1.06869411e+00,
       -8.15658723e-01,  4.78299057e-02, -1.80950446e+00, -6.20742770e-01,
       -1.86613150e+00, -1.51463648e+00, -4.69575729e-01, -1.84533347e-01,
       -1.53003976e+00, -1.06588493e+00, -1.70079255e+00, -9.78608681e-01,
       -1.66542749e+00, -1.76103533e+00, -1.53687642e+00, -1.76459685e+00,
       -8.03571987e-01, -1.15443131e+00, -1.63071412e+00, -1.12482561e+00,
       -1.18766346e-01, -

### 9.2.1 가우시안 혼합을 사용한 이상치 탐지


In [76]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities,4)
anomalies = X[densities < density_threshold]

### 9.2.2 클러스터 개수 선택
* BIC, AIC를 최소화하는 모델을 찾음

In [77]:
gm.bic(X)

2770.974768960084

In [78]:
gm.aic(X)

2687.5429292173876

### 9.2.3 베이즈 가우시안 혼합 모델
* 최적의 클러스터 개수를 수동으로 찾지않고 불필요한 클러스터의 가중치를 0으로 혹은 0에 가깝게 만드는 **BayesianGaussianMixture**사용

In [79]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

array([0.11, 0.12, 0.14, 0.13, 0.12, 0.13, 0.11, 0.13, 0.  , 0.  ])

### 9.2.4 이상치 탐지와 특이치 탐지를 위한 알고리즘
* PCA: 차원 축소 기법
* Fast_MCD: 데이터셋 정제
* 아이솔레이션 포레스트: 무작위로 성장한 결정 트리로 구성된 랜덤 포레스트 생성
* LOF: 샘플 주위 밀도와 이웃 주위 밀도 비교
* one-class SVM: 특이치 탐지에 잘 맞음